# **Statistical learning final project**: dataset selection and preprocessing

### Daniel A.
### UID: 100444499

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pycountry
import json

### Importing data

In [2]:
# Importing the raw data
raw_data = pd.read_csv('./data/raw/wb_raw_data.csv')

### Initial modifications and selection of range of rows

In [3]:
# excluding aggregates
codes_to_exclude = raw_data.iloc[2593:2640,3].values

# filtering the dataset
data = raw_data[~(raw_data['Country Code'].isin(codes_to_exclude))] 

# removing final diagnostic columns
data = data[~(data['Time'].isna()) & ~(data['Time Code'].isna())]

# converting year column to integer
data['Time'] = data['Time'].astype(int)

# replacing .. with NAN as the raw data indends this to be a NAN
data = data.replace('..',np.nan)

# sorting values
data = data.sort_values(['Time','Country Name'])

#### Years imported

In [4]:
# checking years we have queried
years = data.Time.unique()
print(years)

[2002 2004 2005 2008 2010 2012 2013 2015 2018 2020]


### Filling NAs with previous year's data

In [5]:
# making a dictionary with the subsets of the main dataframe
dfs = {}
for year in years:
    dfs[year] = data[data['Time'] == year].reset_index(drop=True)

# replacing nans in the 2020 dataframe with previous years data, as 
# the previous years' data still serves us a purpose for the analysis
for year in years:   
    dfs[2020] = dfs[2020].fillna(dfs[year])

### Removing columns with more than 45 NAs

In [6]:
# removing columns where there's too many NANs
cols_to_keep = []
df = {}
for col,val in zip(dfs[2020].columns,dfs[2020].isna().sum()):
    if val < 45:
        cols_to_keep.append(col)
df = dfs[2020][cols_to_keep]

### Removing life expectancy as it is a component of HDI (our target)

In [8]:
# removing life exp
df.drop("Life expectancy at birth, total (years)", axis=1, inplace=True)

### Removing countries with more than 4 NAs

In [9]:
# checking which countries have the most NANs
all_countries = df['Country Name'].values
countries_removed = []
for country,val in zip(all_countries,df.isna().sum(axis=1)):
    # removing at 3 nans per row
    if val > 4:
        countries_removed.append(country)

# finally filtering to remove them
# either way, these countries are mostly dependencies or 
# complex countries to get data from, like North Korea
# so even after imputing, this would probably
# yield unrealistic values
df = df[~(df['Country Name'].isin(countries_removed))]

### Obtaining HDI per country from Wikipedia

In [10]:
# scraping the wikipedia page for list of countries by human development index
hdis_page = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index').text
soup = BeautifulSoup(hdis_page,'lxml')


# get the HDI table
table = soup.find('table',{'class':'wikitable sortable'})
links = table.findAll('a')
tds = table.findAll('td')
countries, hdis = [], []

# going through the links to find the countries wikipedia has data for
for link in links:
    countries.append(link.get('title'))

# cleaning countriesif not string
countries = [x for x in countries if x != None]

# finding the HDIs and appending them to the list hdi
for td in tds:
    try:
        num = float(td.text)
        if str(num)[0:2] == '0.':
            hdis.append(num)
    except:
        continue

# making a dict with the values
countries = {cont:['',hdi] for cont,hdi in zip(countries,hdis)}

# using the same approach for the codes for easier matching later on
codes_page = requests.get('https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3').text
soup = BeautifulSoup(codes_page,'lxml')

# get the codes table
table = soup.find('div',{'class':'plainlist'})
# scrape list elements and add them to the dict
lis = table.findAll('li')
li_lists = []
for li in lis:
    try:
        countries[li.find('a').get('title')][0] = li.find('span').text
    except:
        continue

# making the values tuples
countries = {cont:tuple(val) for cont,val in countries.items()}

### Adding HDI to the dataframe

In [11]:
# adding the HDI to the dataframe for the countries which we have in the wikipedia list
df['HDI'] = np.nan
matched_countries = []
missing_codes = []
for code,hdi in countries.values():
    df.loc[df['Country Code'] == code,'HDI'] = hdi

# finally keepiTRUEng this dataframe
df = df[df['Country Code'].isin([x[0] for x in countries.values()])].reset_index(drop=True)

### Fixing columns' types and names

In [12]:
# modifying columns that are supposed to be numeric
cols = list(set(list(df.columns)) - set(['index', 'Time', 'Time Code', 'Country Name', 'Country Code']))
for col in cols:
    df[col] = df[col].astype(float)

# new column names
colnames = ['year','year_code','country_name',
            'country_code','gdp','foreign_inv_inflows',
            'exports_perc_gdp','inflation_perc',
            'education_years','education_perc_gdp','gds_perc_gdp',
            'gross_savings_perc_gdp','int_tourism_arrivals',
            'int_tourism_receipts','gni','perc_internet_users',
            'hdi']

# original colnames : colnames dict
col_dict = dict(zip(df.columns,colnames))

# renaming columns as the names are too long
df.columns = colnames

### Constructing categorical target variable

In [13]:
# function to construct target variable
def hdi(x):
    conds = {
        'very high':x >= 0.8,
        'high':x >= 0.7 and x <= 0.799,
        'medium':x >= 0.55 and x <= 0.699,
        'low':x <0.55
    }
    for cat,cond in conds.items(): 
        if cond == True:
            return cat 

df['hdi_cat'] = df['hdi'].apply(lambda x: hdi(x))

### Exporting the data from Python into R to impute the few missing values left

In [14]:
# saving the dataset before imputing
df.to_csv('./data/preproc_outputs/data_before_imp.csv')

# dumping the col information into a json
col_dict['Human development index as a category (1-4)'] = 'hdi_cat'

with open('./data/preproc_outputs/columns.json','w') as f:
    f.write(json.dumps(col_dict))

In [1]:
# importing mice library to impute the data
library(mice)

# reading the csv outputted earlier in the IRkernel to impute the data
df <- read.csv('./data/preproc_outputs/data_before_imp.csv')
df <- df[,2:length(names(df))]

# instantiating the imputer and imputing the data
imp = mice(df, m = 5, method = "cart")
df = complete(imp)

# saving csv for reference
write.csv(df,'./data/data.csv')


Attaching package: ‘mice’


The following object is masked from ‘package:stats’:

    filter


The following objects are masked from ‘package:base’:

    cbind, rbind


Warning message:
“Number of logged events: 6”



 iter imp variable
  1   1  foreign_inv_inflows  exports_perc_gdp  inflation_perc  education_years  education_perc_gdp  gds_perc_gdp  gross_savings_perc_gdp  int_tourism_arrivals  int_tourism_receipts
  1   2  foreign_inv_inflows  exports_perc_gdp  inflation_perc  education_years  education_perc_gdp  gds_perc_gdp  gross_savings_perc_gdp  int_tourism_arrivals  int_tourism_receipts
  1   3  foreign_inv_inflows  exports_perc_gdp  inflation_perc  education_years  education_perc_gdp  gds_perc_gdp  gross_savings_perc_gdp  int_tourism_arrivals  int_tourism_receipts
  1   4  foreign_inv_inflows  exports_perc_gdp  inflation_perc  education_years  education_perc_gdp  gds_perc_gdp  gross_savings_perc_gdp  int_tourism_arrivals  int_tourism_receipts
  1   5  foreign_inv_inflows  exports_perc_gdp  inflation_perc  education_years  education_perc_gdp  gds_perc_gdp  gross_savings_perc_gdp  int_tourism_arrivals  int_tourism_receipts
  2   1  foreign_inv_inflows  exports_perc_gdp  inflation_perc  educat